## Loading and cleaning

In [12]:
import sys
sys.path.insert(0, '../repository')

In [13]:
import database
import pandas as pd
import numpy as np
import glob
import re

In [14]:
conn = database.DatabaseConnection(db_name='doing-bussiness').connection

In [15]:
f = open("../repository/DUMP/ISO CODE.csv", "r")
iso_codes = pd.read_csv(f)
iso_codes['id'] = pd.Series(range(len(iso_codes)), name='id')

f = open("../repository/RAW/WB - Doing Business.csv", mode="r", encoding="utf-8")
df = pd.read_csv(f, sep=";", index_col=False)
df = df.dropna(how="all").convert_dtypes()

In [16]:
df.columns = [re.sub(r"\(.*\)|'", "", c).strip() for c in df.columns]
df.columns = [re.sub(r"-| ", "_", c).lower() for c in df.columns]

In [17]:
df = df.merge(iso_codes.iloc[:, [-1, 0, 2]], left_on="country_code", right_on="alpha-3")
df = df.drop(axis=1, columns=["economy", "name", "alpha-3", "region"])
df = df.rename(columns={"id": "country_id", "db_year": "year"})
df = df[["country_id", "year", *df.columns]]
df = df.drop(columns=["country_code"])
df = df.loc[:, ~df.columns.duplicated()].copy()


In [18]:
# df.to_csv('../repository/DUMP/Doing Business/DB - Freedom.csv', sep=';', index=False, decimal='.')
df.to_sql('freedom', con=conn, if_exists='replace', index_label='id')

59